In [1]:
import pandas as pd
import numpy as np
import statistics as stat
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("white")
import statistics as stat

In [2]:
data = pd.read_csv('/home/smansou2/Desktop/experimentAnalysis/scdb-test-metrics.csv', names = ['Event', 'Time', 'Tx_type', 'Tx_id', 'Accept_id', 'time'])

In [3]:
grouped_data = data.groupby('Tx_id')
tx_id = list(grouped_data.groups.keys())
len(tx_id)

5834

In [4]:
#new data frame to get the count of each transaction type
data_unique_ids = data.drop_duplicates(subset = ['Tx_id'])

adv_count = 0
adv_ids = []
for i in range(0, len(data_unique_ids)):
    if data_unique_ids.iloc[i]['Tx_type'] == 'ADV':
        adv_ids.append(data_unique_ids.iloc[i]['Tx_id'])
        adv_count += 1   


print(f"#ADVs =  {adv_count}")

#ADVs =  1497


In [5]:
df_test = grouped_data.get_group(adv_ids[0])
df_test

,Event,Time,Tx_type,Tx_id,Accept_id,time
36,received_tx,2835,ADV,ee527bb4380c78b67b49c9736ca68c37c7bb6fb9edff65...,None,1.733811e+09
37,before_tendermint,2873,ADV,ee527bb4380c78b67b49c9736ca68c37c7bb6fb9edff65...,None,1.733811e+09
38,check_tx,2918,ADV,ee527bb4380c78b67b49c9736ca68c37c7bb6fb9edff65...,None,1.733811e+09
39,deliver_tx,2969,ADV,ee527bb4380c78b67b49c9736ca68c37c7bb6fb9edff65...,None,1.733811e+09
40,end_block,2972,ADV,ee527bb4380c78b67b49c9736ca68c37c7bb6fb9edff65...,None,1.733811e+09
41,commit_tx,2978,ADV,ee527bb4380c78b67b49c9736ca68c37c7bb6fb9edff65...,None,1.733811e+09


In [6]:
df_test = grouped_data.get_group(adv_ids[1])
df_test

,Event,Time,Tx_type,Tx_id,Accept_id,time
42,received_tx,2818,ADV,9d9c44613fba6921c29c909129dee40f5d2fed56e8e9fe...,None,1.733811e+09
43,before_tendermint,2851,ADV,9d9c44613fba6921c29c909129dee40f5d2fed56e8e9fe...,None,1.733811e+09
44,check_tx,2903,ADV,9d9c44613fba6921c29c909129dee40f5d2fed56e8e9fe...,None,1.733811e+09
45,deliver_tx,2971,ADV,9d9c44613fba6921c29c909129dee40f5d2fed56e8e9fe...,None,1.733811e+09
46,end_block,2976,ADV,9d9c44613fba6921c29c909129dee40f5d2fed56e8e9fe...,None,1.733811e+09
47,commit_tx,2984,ADV,9d9c44613fba6921c29c909129dee40f5d2fed56e8e9fe...,None,1.733811e+09


In [7]:
time_all_committed_advs = {}
time_passed_advs = []
validated_time = []
accepted_time = []
committed_time = []

adv_underfit = []
adv_underfit_dict = {}

adv_overfit = []
adv_overfit_dict = {}

for i in range(0, len(tx_id)):
    temp_list = []
    if "ADV" == grouped_data.get_group(tx_id[i])["Tx_type"].to_list()[0]:
        temp_list = grouped_data.get_group(tx_id[i])["Time"].to_list()
        if len(temp_list) == 6 and grouped_data.get_group(tx_id[i])["Event"].to_list()[5] == "commit_tx":
            
            actual_time = max(temp_list) - min(temp_list)
            time_all_committed_advs[tx_id[i]] = actual_time
            time_passed_advs.append(actual_time)
            
            df = grouped_data.get_group(tx_id[i])
            received_time = df[df['Event'] == 'received_tx']['Time'].to_list()[0]
            before_tendermint_time = df[df['Event'] == 'before_tendermint']['Time'].to_list()[0]
            
            validated = before_tendermint_time - received_time
            validated_time.append(validated)
            
            check_time = df[df['Event'] == 'check_tx']['Time'].to_list()[0]
            accepted = check_time - received_time
            accepted_time.append(accepted)
            
            commit_time =  df[df['Event'] == 'commit_tx']['Time'].to_list()[0]
            committed = commit_time - received_time
            committed_time.append(committed)
        elif len(temp_list) < 6:
            adv_underfit.append(temp_list)
            adv_underfit_dict[tx_id[i]] = temp_list
        elif 7 >= len(temp_list) > 6 :
            df_check = grouped_data.get_group(tx_id[i])
            if len(df_check[df_check['Event'] == 'check_tx']['Time'].to_list()) > 1:
                check_time = max(df_check[df_check['Event'] == 'check_tx']['Time'].to_list())
            else:
                check_time = df_check[df_check['Event'] == 'check_tx']['Time'].to_list()[0] 
            
            actual_time = max(temp_list) - min(temp_list)
            time_all_committed_advs[tx_id[i]] = actual_time
            time_passed_advs.append(actual_time)
            
            df = grouped_data.get_group(tx_id[i])
            received_time = df[df['Event'] == 'received_tx']['Time'].to_list()[0]
            before_tendermint_time = df[df['Event'] == 'before_tendermint']['Time'].to_list()[0]
            
            validated = before_tendermint_time - received_time
            validated_time.append(validated)
           
            accepted = check_time - received_time
            accepted_time.append(accepted)
            
            commit_time =  df[df['Event'] == 'commit_tx']['Time'].to_list()[0]
            committed = commit_time - received_time
            committed_time.append(committed)
            
            adv_overfit.append(actual_time)
            adv_overfit_dict[tx_id[i]] = temp_list

In [8]:
adv_overfit_dict

{'008dcdd763f68a47814965d0d9ecd00ddf2eacbf7d413a4572e49a86fca14cfc': [55,
  4569,
  549872,
  4032992,
  6274434,
  7121364,
  7121607],
 '009fe7ef4a1ec8083de19fd1ce25295233de94e9b339d41276fe1a353c547890': [43,
  3566,
  544412,
  4027526,
  6268357,
  7129771,
  7130014],
 '00e2bbeb75101e3ee96bcf70f922c3e9fa9a15f2c058d41f4c8f8dd7d15dabb3': [41,
  2427,
  33207,
  1008193,
  4194532,
  5082534,
  5082670],
 '00f0ca1bff31445f55f80dad0c6a505f13b32ee08915ae8fb94cd964e719ff91': [46,
  3456,
  145257,
  3598391,
  5831651,
  7695264,
  7695513],
 '012728c30a19306b14bd4b2e39afc8d93ace0a51194d5694690265606786ba5c': [46,
  3175,
  276318,
  3783458,
  6018675,
  7447199,
  7447444],
 '016584fa2b9f7cd98c5271d9de13ee30350a5a14a4834aca4b294bef02446c7a': [43,
  2306,
  28773,
  995826,
  4172514,
  5114065,
  5114202],
 '027513d621056b775ddc78fefef8e0b3626763d687688ce6d2c3cb2e227c456e': [72,
  8420,
  499375,
  3946916,
  6204787,
  7211761,
  7212004],
 '0283705a8dd6a83b7b9104944b500dede7ed766c95

In [9]:
len(time_passed_advs)

972

In [10]:
len(adv_overfit)

964

In [11]:
stat.mean(time_passed_advs)

6355112.740740741

In [12]:
stat.mean(adv_overfit)

6406837.652489627

In [13]:
# Analysis of CREATE Transaction
adv_mean = "{:.2f}".format(stat.mean(time_passed_advs))
adv_median = stat.median(time_passed_advs)
adv_max = max(time_passed_advs)
adv_min = min(time_passed_advs)
adv_std_dev = "{:.2f}".format(stat.stdev(time_passed_advs))

print(adv_mean)
print(adv_median)
print(adv_max)
print(adv_min)
print(adv_std_dev)
print(f"total advs = {len(time_passed_advs)}")

adv_dict = {
               'min': adv_min,
               'max': adv_max,
               'average': adv_mean,
               'median' : adv_median,
               'standard_deviation' : adv_std_dev,
               '# of txs' : len(time_passed_advs)}

6355112.74
6899762.5
7978246
143
1346937.45
total advs = 972


In [14]:
results = pd.DataFrame(adv_dict, index = ['ADV'])
results.to_csv('adv_analysis.csv', sep=',', encoding='utf-8')
results

,min,max,average,median,standard_deviation,# of txs
ADV,143,7978246,6355112.74,6899762.5,1346937.45,972
